In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from sklearn.model_selection import train_test_split

C:\Users\EduardCP\AppData\Roaming\Python\Python310\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Initialize Spark Session
spark = SparkSession.builder.appName("MLBinaryClassifier").getOrCreate()

# Load all train-X.csv files
train_files = [f"../data/train-{i}.csv" for i in range(1, 9)]
df = spark.read.csv(train_files, header=True, inferSchema=True)

# Load additional data
directing_df = spark.read.json("../data/directing.json")
writing_df = spark.read.json("../data/writing.json")

# Merge additional information if a common key exists
if 'movie' in df.columns:
    df = df.join(directing_df, 'movie', 'left')
    df = df.join(writing_df, 'movie', 'left')

# Handle categorical data
categorical_cols = [col_name for col_name, dtype in df.dtypes if dtype == "string"]
for col_name in categorical_cols:
    indexer = StringIndexer(inputCol=col_name, outputCol=f"{col_name}_index").setHandleInvalid("keep")
    df = indexer.fit(df).transform(df)
    df = df.drop(col_name)  # Drop original categorical columns

# Convert label column to numeric
df = df.withColumn("label", col("label").cast("double"))

# Handle missing values
df = df.fillna(0)

# Define features assembler
feature_cols = [col_name for col_name in df.columns if col_name != "label"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df = assembler.transform(df)

# Standardize numeric features
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=False)
scaler_model = scaler.fit(df)
df = scaler_model.transform(df)

# Split data
train_df, val_df = df.randomSplit([0.8, 0.2], seed=42)

# Train a model
rf = RandomForestClassifier(featuresCol="scaled_features", labelCol="label", numTrees=100, seed=42)
model = rf.fit(train_df)

# Evaluate the model
y_pred = model.transform(val_df)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(y_pred)

print("Model Accuracy:", accuracy)


Model Accuracy: 0.7238903394255874
